# Analisi EDA per i dataset a disposizione

Si realizza un'analisi esplorativa dei dati al fine di capire quali sono i processi di trasformazione da applicare ai dataset a disposizione al fine di realizzare una corretta pipeline E2E su AWS.

In [1]:
import pandas as pd
import numpy as np

In [2]:
btc_prices = r"C:\Users\marco\Desktop\Corso_DE\Corso_AWS\progetto_finale\dataset\BTC_EUR_Historical_Data.csv"
btc_trend = r"C:\Users\marco\Desktop\Corso_DE\Corso_AWS\progetto_finale\dataset\google_trend_bitcoin.csv"
xmr_prices = r"C:\Users\marco\Desktop\Corso_DE\Corso_AWS\progetto_finale\dataset\XMR_EUR Kraken Historical Data.csv"
xmr_trend = r"C:\Users\marco\Desktop\Corso_DE\Corso_AWS\progetto_finale\dataset\google_trend_monero.csv"

## Prezzi bitcoin

In [3]:
btc_df = pd.read_csv(btc_prices, parse_dates=['Date'], thousands=",")
btc_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-03-12,65619.5,65860.0,66135.3,65200.0,0.19K,-0.37%
1,2024-03-11,65859.9,63074.2,66600.0,61517.2,1.88K,4.42%
2,2024-03-10,63074.3,62613.1,63930.6,62400.0,0.72K,0.68%
3,2024-03-09,62649.9,62425.2,62781.1,62124.5,0.37K,0.38%
4,2024-03-08,62413.8,61130.1,63999.0,60483.8,1.61K,2.10%


In [4]:
btc_df.count()

Date        1887
Price       1887
Open        1887
High        1887
Low         1887
Vol.        1887
Change %    1887
dtype: int64

In [5]:
btc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1887 non-null   datetime64[ns]
 1   Price     1887 non-null   float64       
 2   Open      1887 non-null   float64       
 3   High      1887 non-null   float64       
 4   Low       1887 non-null   float64       
 5   Vol.      1887 non-null   object        
 6   Change %  1887 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 103.3+ KB


No null or NaN values.

In [6]:
btc_df[btc_df['Price'] == -1].count()

Date        18
Price       18
Open        18
High        18
Low         18
Vol.        18
Change %    18
dtype: int64

Ci sono alcuni valori della colonna Price che non danno un'informazione effettiva per le analisi. Si può pensare di sostituirle con la media mobile delle ultime 10 osservazioni, trattandosi di valori di prezzi su più anni. In questo modo non si introducono elevate fluttuazioni del valore per il giorno specifico.

Anche se non necessario ai fini del progetto attuale, si realizza un preprocessing anche delle altre colonne presenti nel dataset, al fine di sfruttare la pipeline in costruzione, per ritrovarsi dei dati pretrattati da salvare sul Data Catalog di Glue, per rendere tali dati disponibili per altri servizi AWS e da cui poter estrarre dei possibili insights.

In [7]:
btc_df.rename(columns={"Vol.":"Vol.[K]"}, inplace=True)
btc_df.head()

,Date,Price,Open,High,Low,Vol.[K],Change %
0,2024-03-12,65619.5,65860.0,66135.3,65200.0,0.19K,-0.37%
1,2024-03-11,65859.9,63074.2,66600.0,61517.2,1.88K,4.42%
2,2024-03-10,63074.3,62613.1,63930.6,62400.0,0.72K,0.68%
3,2024-03-09,62649.9,62425.2,62781.1,62124.5,0.37K,0.38%
4,2024-03-08,62413.8,61130.1,63999.0,60483.8,1.61K,2.10%


In [8]:
btc_df['Vol.[K]'] = btc_df['Vol.[K]'].map(lambda x:x.replace('K',''))
btc_df.head()

,Date,Price,Open,High,Low,Vol.[K],Change %
0,2024-03-12,65619.5,65860.0,66135.3,65200.0,0.19,-0.37%
1,2024-03-11,65859.9,63074.2,66600.0,61517.2,1.88,4.42%
2,2024-03-10,63074.3,62613.1,63930.6,62400.0,0.72,0.68%
3,2024-03-09,62649.9,62425.2,62781.1,62124.5,0.37,0.38%
4,2024-03-08,62413.8,61130.1,63999.0,60483.8,1.61,2.10%


In [9]:
btc_df['Vol.[K]'] = btc_df['Vol.[K]'].map(lambda x: float(x))


In [10]:
btc_df['Change %'] = btc_df['Change %'].map(lambda x:x.replace('%',''))
btc_df['Change %'] = btc_df['Change %'].map(lambda x: float(x))
btc_df.head()

,Date,Price,Open,High,Low,Vol.[K],Change %
0,2024-03-12,65619.5,65860.0,66135.3,65200.0,0.19,-0.37
1,2024-03-11,65859.9,63074.2,66600.0,61517.2,1.88,4.42
2,2024-03-10,63074.3,62613.1,63930.6,62400.0,0.72,0.68
3,2024-03-09,62649.9,62425.2,62781.1,62124.5,0.37,0.38
4,2024-03-08,62413.8,61130.1,63999.0,60483.8,1.61,2.10


In [11]:
btc_df.tail()

,Date,Price,Open,High,Low,Vol.[K],Change %
1882,2019-01-16,3164.7,3149.2,3228.9,3140.2,5.52,0.49
1883,2019-01-15,3149.2,3197.6,3214.3,3119.8,5.36,-1.51
1884,2019-01-14,3197.6,3075.0,3238.8,3068.1,7.29,4.02
1885,2019-01-13,3074.0,3168.1,3188.0,3052.2,3.52,-2.97
1886,2019-01-12,3168.1,3175.0,3193.2,3130.0,2.36,-0.21


In [12]:
btc_df.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Vol.[K]            float64
Change %           float64
dtype: object

## Bitcoin trend

In [13]:
btc_trend_df = pd.read_csv(btc_trend, parse_dates=['Settimana'])
btc_trend_df.head()

,Settimana,interesse bitcoin
0,2019-03-17,9
1,2019-03-24,9
2,2019-03-31,15
3,2019-04-07,13
4,2019-04-14,11


In [14]:
btc_trend_df.tail()

,Settimana,interesse bitcoin
255,2024-02-04,20
256,2024-02-11,29
257,2024-02-18,22
258,2024-02-25,43
259,2024-03-03,52


In [15]:
btc_trend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Settimana          260 non-null    datetime64[ns]
 1   interesse bitcoin  260 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.2 KB


In [16]:
btc_trend_df.count()

Settimana            260
interesse bitcoin    260
dtype: int64

This dataset need just to be normalised as it follows.

In [17]:
btc_trend_df.rename(columns={"Settimana":"Week", "interesse bitcoin":"Google_trend_index"}, inplace=True)
btc_trend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Week                260 non-null    datetime64[ns]
 1   Google_trend_index  260 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.2 KB


## Prezzi Monero

In [18]:
xmr_df = pd.read_csv(xmr_prices, parse_dates=['Date'], thousands=",")
xmr_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-03-12,133.29,132.67,134.41,130.11,1.28K,0.45%
1,2024-03-11,132.69,133.76,134.38,130.06,5.36K,-0.80%
2,2024-03-10,133.76,131.40,135.65,130.26,2.22K,1.80%
3,2024-03-09,131.40,134.70,136.09,131.33,2.57K,-2.45%
4,2024-03-08,134.70,131.74,138.47,130.46,4.54K,2.25%


In [19]:
xmr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1898 non-null   datetime64[ns]
 1   Price     1898 non-null   float64       
 2   Open      1898 non-null   float64       
 3   High      1898 non-null   float64       
 4   Low       1898 non-null   float64       
 5   Vol.      1895 non-null   object        
 6   Change %  1898 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 103.9+ KB


Vol. column has some NaN values that needs to be managed. It is possibile to impute the missing values with the rolling mean of the last 10 observations.

In [20]:
xmr_df['Vol.'] = xmr_df['Vol.'].astype(str)
xmr_df['Vol.'] = xmr_df['Vol.'].replace({'nan': np.nan, '': np.nan})
xmr_df.rename(columns={"Vol.":"Vol.[K]"}, inplace=True)
#xmr_df['Vol.[K]'] = xmr_df['Vol.[K]'].map(lambda x:x.replace('K',''))
#xmr_df['Vol.[K]'] = xmr_df['Vol.[K]'].map(lambda x: float(x))
xmr_df['Vol.[K]'] = xmr_df['Vol.[K]'].str.replace('K', '', regex=False).astype(float)
xmr_df['Vol.[K]'] = xmr_df['Vol.[K]'].fillna(xmr_df['Vol.[K]'].rolling(window=10, min_periods=1).mean())


In [21]:
print(xmr_df[xmr_df['Date']== pd.to_datetime('2020-07-10')])

           Date  Price   Open   High    Low   Vol.[K] Change %
1341 2020-07-10  60.03  59.75  60.03  58.72  4.505556    2.23%


In [22]:
xmr_df.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Vol.[K]            float64
Change %            object
dtype: object

In [23]:
xmr_df['Change %'] = xmr_df['Change %'].map(lambda x:x.replace('%',''))
xmr_df['Change %'] = xmr_df['Change %'].map(lambda x: float(x))
xmr_df.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Vol.[K]            float64
Change %           float64
dtype: object

In [24]:
xmr_df.head()

,Date,Price,Open,High,Low,Vol.[K],Change %
0,2024-03-12,133.29,132.67,134.41,130.11,1.28,0.45
1,2024-03-11,132.69,133.76,134.38,130.06,5.36,-0.80
2,2024-03-10,133.76,131.40,135.65,130.26,2.22,1.80
3,2024-03-09,131.40,134.70,136.09,131.33,2.57,-2.45
4,2024-03-08,134.70,131.74,138.47,130.46,4.54,2.25


In [25]:
xmr_df[xmr_df['Price'] == -1].count()

Date        22
Price       22
Open        22
High        22
Low         22
Vol.[K]     22
Change %    22
dtype: int64

Also for XMR data, there are 22 days where the Price value needs to be managed: it is possible to impute this value with the rolling mean of the last 10 days.

In [26]:
xmr_df['rolling_mean'] = xmr_df['Price'].rolling(window=10, min_periods=1).mean()
xmr_df.loc[xmr_df['Price'] == -1, 'Price'] = xmr_df.loc[xmr_df['Price'] == -1, 'rolling_mean']
xmr_df.drop(columns=['rolling_mean'], inplace=True)

In [27]:
xmr_df[xmr_df['Price'] == -1].count()

Date        0
Price       0
Open        0
High        0
Low         0
Vol.[K]     0
Change %    0
dtype: int64

## Monero trend

In [28]:
xmr_trend_df = pd.read_csv(xmr_trend)
xmr_trend_df.head()

,Settimana,Monero_interesse
0,2019-03-17,26
1,2019-03-24,23
2,2019-03-31,28
3,2019-04-07,25
4,2019-04-14,23


In [29]:
xmr_trend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Settimana         260 non-null    object
 1   Monero_interesse  260 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.2+ KB


In [30]:
xmr_trend_df.count()

Settimana           260
Monero_interesse    260
dtype: int64

In [31]:
xmr_trend_df.rename(columns={"Settimana":"Week", "Monero_interesse":"Google_trend_index"}, inplace=True)
xmr_trend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Week                260 non-null    object
 1   Google_trend_index  260 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.2+ KB


## Conclusioni

I dataset relativi ai prezzi di entrambe le criptovalute hanno delle strutture simili, che condividono le analoghe criticità:
- ci sono alcuni valori della colonna Prezzi pari a -1, essendo un valore che di fatto non restituisce un'informazione valida per le analisi esso è stato sostituito con la media mobile del prezzo delle ultime 10 osservazioni;
- è stato realizzato un preprocessing simile per entrambi i dataset, trattando anche delle colonne non utili ai fini delle richieste progettuali, ma comunque trattate per poter salvare il dato elaborato all'interno del data catalog di Glue, al fine di renderli disponibili per altri servizi e comunque fonte d'informazioni da cui trarre ulteriori insights.

Per quanto riguarda i dataset dei trend non c'è molto da dire se non normalizzare i nomi delle colonne.

Di seguito si riportano dei metodi che possono essere usati nei job Glue per trattare il dataset, viste le evidenze emerse in quest'analisi preliminare del dataset.

In [32]:
def price_substitution(df):
    df['rolling_mean'] = df['Price'].rolling(window=10, min_periods=1).mean()
    df.loc[df['Price'] == -1, 'Price'] = df.loc[df['Price'] == -1, 'rolling_mean']
    df.drop(columns=['rolling_mean'], inplace=True)
    
    return df

def price_dataset_preprocessing(price_dataset_file:str):
    #import data in Pandas Dataframe
    df = pd.read_csv(price_dataset_file, parse_dates=['Date'], thousands=',')

    ###### Vol. column preprocessing######################################
    
    #string conversion and NaN replace
    df['Vol.'] = df['Vol.'].astype(str).replace({'nan': np.nan, '': np.nan})
    df.rename(columns={"Vol.":"Vol.[K]"}, inplace=True)
    #float conversion
    df['Vol.[K]'] = df['Vol.[K]'].str.replace('K', '', regex=False).astype(float)
    #nan values are replaced with the rolling mean of the last 10 observations
    df['Vol.[K]'] = df['Vol.[K]'].fillna(df['Vol.[K]'].rolling(window=10, min_periods=1).mean())

    ######## Change % col. preprocessing ##########################
    df['Change %'] = df['Change %'].map(lambda x:x.replace('%',''))
    df['Change %'] = df['Change %'].map(lambda x: float(x))

    return df

def label_normalizer(trend_dataset_file:str):
    df = pd.read_csv(trend_dataset_file, parse_dates=['Settimana'])
    columns = df.columns
    df.rename(columns={columns[0]:"Week", columns[1]:"Google_trend_index"}, inplace=True)
    return df